1. make a imbalanced dataset and use both class weight and BalancedRandomForestRegressor
2. Take iris dataset
   Use randomforestregressor use OOB
   do it with bootstraping random subspace, and random pathces
   use hyperparameters which are specific to random forest
   do feature selection using random forest
   Feature importance based on mean decrease in impurity

1. make a imbalanced dataset and use both class weight and BalancedRandomForestRegressor

In [3]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=5000, n_features=10, n_classes=2, weights=[0.9, 0.1], random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

rf = RandomForestClassifier(n_estimators=100, class_weight='balanced')
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)

brf = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
brf.fit(X_train, y_train)
brf_preds = brf.predict(X_test)

print("=== Logistic Regression (class_weight='balanced') ===")
print(classification_report(y_test, rf_preds))

print("=== Balanced Random Forest ===")
print(classification_report(y_test, brf_preds))

=== Logistic Regression (class_weight='balanced') ===
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       894
           1       0.87      0.64      0.74       106

    accuracy                           0.95      1000
   macro avg       0.92      0.82      0.86      1000
weighted avg       0.95      0.95      0.95      1000

=== Balanced Random Forest ===
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       894
           1       0.72      0.82      0.77       106

    accuracy                           0.95      1000
   macro avg       0.85      0.89      0.87      1000
weighted avg       0.95      0.95      0.95      1000



In [4]:
from sklearn.datasets import load_iris 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
import numpy as np

iris = load_iris()
X,y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

baseline_rf = RandomForestClassifier(oob_score=True, random_state=42)
baseline_rf.fit(X_train, y_train)

print(f"OOB score: {baseline_rf.oob_score_:.3f}")

OOB score: 0.943


In [7]:
# 4. Hyperparameter Grid
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'max_samples': [0.5, 0.75, None],
    'max_depth': [None, 5, 10, 20]
}

rf = RandomForestClassifier(oob_score=True, random_state=42)
random_search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=10, cv=3, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

best_rf = random_search.best_estimator_
print(f'\n Tuned OOB Score: {best_rf.oob_score_:.3f}')
print("Best Parameters: ", random_search.best_params_)

y_pred = best_rf.predict(X_test)
print("\n Classification Report: \n", classification_report(y_test, y_pred))


 Tuned OOB Score: 0.943
Best Parameters:  {'n_estimators': 50, 'max_samples': None, 'max_features': 'sqrt', 'max_depth': None, 'bootstrap': True}

 Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
24 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_forest.py", line 431, in fit
    raise ValueErr